## Neural Network Hyperparameter Tuning with Keras Tuner

In [1]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

11490434/11490434 [==============================] - 0s 0us/step


(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train.min(), X_train.max()

(0.0, 1.0)

In [5]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
  hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=100)
  hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=100)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
  model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

In [6]:
import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='dir',
                     project_name='x')

In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [8]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 01m 48s]
val_accuracy: 0.9541666507720947

Best val_accuracy So Far: 0.9793333411216736
Total elapsed time: 00h 24m 59s


In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [10]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2,
                    callbacks=[stop_early])

Epoch 1/50
1500/1500 [==============================] - 16s 10ms/step - loss: 0.3465 - accuracy: 0.9074 - val_loss: 0.1709 - val_accuracy: 0.9523
Epoch 2/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.1439 - accuracy: 0.9593 - val_loss: 0.1243 - val_accuracy: 0.9659
Epoch 3/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0974 - accuracy: 0.9715 - val_loss: 0.1046 - val_accuracy: 0.9688
Epoch 4/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0717 - accuracy: 0.9798 - val_loss: 0.0960 - val_accuracy: 0.9716
Epoch 5/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0542 - accuracy: 0.9845 - val_loss: 0.0863 - val_accuracy: 0.9745
Epoch 6/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0403 - accuracy: 0.9885 - val_loss: 0.0782 - val_accuracy: 0.9761
Epoch 7/50
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0301 - accuracy: 0.9920 - val_loss: 0.0837 -

In [11]:
history.history

{'loss': [0.3465227484703064,
  0.143879696726799,
  0.09736238420009613,
  0.07168809324502945,
  0.05415351316332817,
  0.040272828191518784,
  0.030102109536528587,
  0.0227852500975132,
  0.016944991424679756,
  0.012930797412991524,
  0.009691490791738033,
  0.0073501598089933395,
  0.006864709779620171,
  0.005705483257770538],
 'accuracy': [0.9073958396911621,
  0.9592708349227905,
  0.9715208411216736,
  0.979812502861023,
  0.9844791889190674,
  0.9885416626930237,
  0.9919999837875366,
  0.9942708611488342,
  0.9958541393280029,
  0.996916651725769,
  0.9978541731834412,
  0.9983958601951599,
  0.9985625147819519,
  0.9986458420753479],
 'val_loss': [0.17086701095104218,
  0.12426631897687912,
  0.10459719598293304,
  0.0960000678896904,
  0.08628085255622864,
  0.07819623500108719,
  0.08372271060943604,
  0.07820059359073639,
  0.07661598920822144,
  0.09314414113759995,
  0.07656877487897873,
  0.08118998259305954,
  0.08153584599494934,
  0.08790067583322525],
 'val_accur

In [12]:
import pandas as pd

pd.DataFrame(history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.346523,0.907396,0.170867,0.952333
1,0.143880,0.959271,0.124266,0.965917
2,0.097362,0.971521,0.104597,0.968833
3,0.071688,0.979813,0.096000,0.971583
4,0.054154,0.984479,0.086281,0.974500
5,0.040273,0.988542,0.078196,0.976083
6,0.030102,0.992000,0.083723,0.974917
7,0.022785,0.994271,0.078201,0.976833
8,0.016945,0.995854,0.076616,0.977083
9,0.012931,0.996917,0.093144,0.973250


In [13]:
model